# Tensorflow CNN implementation based on the example code of the documentation
### using opencv and a custom saved model for classification

Note: Use Tensorflow 2.4.1  (pip3 install tensorflow==2.4.1)

### How to use:
- download your model weights folder from your google drive (if you trained using google colab) or make sure your model weights fodler are in the right directory for the "model_name"
- "model_name" has to match your weights folder
-  run all cells. 
- if the device (webcam) can't be opened, try changing the devide_number to either -1, 0, 1 or 2. (This depends if your system has an internal camera or not)
- devide_number can also be used to switch between webcams if more than one is available 

Usefull links: <br>
CNN Model is based on: https://www.tensorflow.org/tutorials/images/classification


## Import needed libraries

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import pathlib

## Setup

In [2]:
img_height = 416
img_width = 416
dim = (img_width, img_height)
model_name = "model_1_750E32B"
devide_number = -1
class_names = ['Hammer', 'Workspace']
rotate_frame = True
#Load Model
model = tf.keras.models.load_model(model_name)

# Webcam Detection

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import numpy as np
import cv2



cap = cv2.VideoCapture(devide_number)
if not (cap.isOpened()):
    print("Could not open video device")

while(True):    
    # cap frame by frame   
    ret, frame = cap.read()    
    
    # rotate image (if needed), crop image to 416x416 (training data size)
    if (rotate_frame):
        frame = cv2.rotate(frame, cv2.ROTATE_180)
    y = 100
    x = 50
    frame = frame[y:y+img_height, x:x+img_width]
    cv2.imwrite("tmp_hammer.jpg", frame) #safe img -> keras needs the path

    tst_img = pathlib.Path("tmp_hammer.jpg") #load img
    img = keras.preprocessing.image.load_img(tst_img, target_size=(img_height, img_width))

    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])

    #print("{} {:.2f}".format(class_names[np.argmax(score)], 100 * np.max(score)))

    if class_names[np.argmax(score)] == "Hammer":
        cv2.putText(frame, "Hammer in Workspace ({:.2f})".format(100 * np.max(score)), (10, 40), 1, 1, (0,255,0), 2) 
    
    else:
       cv2.putText(frame, "No Hammer in Workspace ({:.2f})".format(100 * np.max(score)), (10, 40), 1, 1, (0,0,255), 2)

    cv2.imshow('Press q to exit', frame)
    #Waits for a user input to quit the application  
    if cv2.waitKey(1) & 0xFF == ord('q'):    
        break

cap.release()
cv2.destroyAllWindows()